<CENTER>
</br>
<p><font size="5">  M2MO & EY - Internship </font></p>
<p><font size="5">  Modelling Forward Initial Margin and Counterparty Risk in Uncleared Derivatives </font></p>
<p><font size="4">  SANGLIER Nathan </font></p>
<p><font size="3"></br>September 2025</font></br></div>
<p><span style="color:blue">nathan.sanglier@fr.ey.com</span>
</p>
</CENTER>

This notebook generates the forward IM profile and error metrics for different hyperparameters/regression techniques related to the models and case studies investigated. 

## <span id="section-0" style="color:#00B8DE"> 0 - Imports </span>

In [11]:
import  time
import  pandas                      as      pd
import  numpy                       as      np
import  matplotlib.pyplot           as      plt
from    backend.utils               import  TimeGrid, seconds_to_minutes
from    backend.pricing_models      import  BlackScholes, YieldCurve, OneFactorHullWhite
from    backend.pricing_engines     import  PutBlackScholes, SwaptionOneFactorHullWhite
from    backend.forward_im_models   import  AnalyticalForwardInitialMargin, GaussianLeastSquaresMonteCarlo, JohnsonLeastSquaresMonteCarlo, JohnsonPercentileMatchingMonteCarlo, NeuralQuantileRegression

In [2]:
np.random.seed(3)
plt.style.use('ggplot')

## <span id="section-1" style="color:#00B8DE"> I - Parameters </span>

In [3]:
num_paths_train = 10000
num_paths_test  = 1000
maturity        = 1
time_grid       = TimeGrid(1/240, maturity)
alpha           = 0.99
mpor            = 1/24

Let us define the settings investigated here.

In [4]:
moment_matching_setting_jlsmc_list = [
    {'id': 0, 'fitting': 'mm', 'method': 'LR',  'ridge': 0, 'basis_type': 'laguerre', 'order': 2, 'regress_mean': True},
    {'id': 1, 'fitting': 'mm', 'method': 'LR',  'ridge': 0, 'basis_type': 'laguerre', 'order': 4, 'regress_mean': True},
    {'id': 2, 'fitting': 'mm', 'method': 'GLM', 'ridge': 0, 'basis_type': 'laguerre', 'order': 2, 'regress_mean': False},
    {'id': 3, 'fitting': 'mm', 'method': 'GLM', 'ridge': 0, 'basis_type': 'laguerre', 'order': 4, 'regress_mean': False},
]

moment_matching_setting_glsmc_list = [
    {'id': 0, 'method': 'LR',   'ridge': 0, 'basis_type': 'canonical', 'order': 2, 'regress_mean': False},
    {'id': 1, 'method': 'LR',   'ridge': 0, 'basis_type': 'canonical', 'order': 2, 'regress_mean': True},
    {'id': 2, 'method': 'GLM',  'ridge': 0, 'basis_type': 'canonical', 'order': 2, 'regress_mean': False},
    {'id': 3, 'method': 'LR',   'ridge': 0, 'basis_type': 'canonical', 'order': 4, 'regress_mean': False},
    {'id': 4, 'method': 'LR',   'ridge': 0, 'basis_type': 'canonical', 'order': 4, 'regress_mean': True},
    {'id': 5, 'method': 'GLM',  'ridge': 0, 'basis_type': 'canonical', 'order': 4, 'regress_mean': False}
]

percentile_matching_setting_list = [
    {'id': 0, 'fitting': 'pm', 'Nnmc': 1000,    'z': 1},
    {'id': 1, 'fitting': 'pm', 'Nnmc': 10000,   'z': 1},
    {'id': 2, 'fitting': 'pm', 'Nnmc': 1000,    'z': 0.524},
    {'id': 3, 'fitting': 'pm', 'Nnmc': 10000,   'z': 0.524},
]

support_values_setting_list = [
    {'id': 0, 'initial nb': 100, 'perc add tails': 0.1, 'add ends': False},
    {'id': 1, 'initial nb': 100, 'perc add tails': 0.1, 'add ends': True}
] 

quantile_function_setting_list = [
    {'id': 0, 'method': 'LR', 'ridge': 0, 'basis_type': 'laguerre', 'order': 2},
    {'id': 1, 'method': 'LR', 'ridge': 0, 'basis_type': 'laguerre', 'order': 4},
    {'id': 2, 'method': 'kNN', 'n_neighbors':1},
    {'id': 3, 'method': 'kNN', 'n_neighbors':3}
]

nn_function_setting_list = [
    {'id': 0, 'batch_size': 1024,   'n_neurons': 4,     'n_hidden_layers': 1, 'optimizer': 'sgd',   'learning_rate': 10**(-2),      'ridge': 10**(-6),      'patience': 3, 'n_epochs': 15}, # tiny SGD baseline (fast & stable)
    {'id': 1, 'batch_size': 1024,   'n_neurons': 10,    'n_hidden_layers': 1, 'optimizer': 'adam',  'learning_rate': 10**(-2),      'ridge': 10**(-6),      'patience': 3, 'n_epochs': 20}, # tiny Adam baseline
    {'id': 2, 'batch_size': 512,    'n_neurons': 8,     'n_hidden_layers': 1, 'optimizer': 'adam',  'learning_rate': 5*10**(-4),    'ridge': 10**(-5),      'patience': 5, 'n_epochs': 20}, # very small net
    {'id': 3, 'batch_size': 256,    'n_neurons': 16,    'n_hidden_layers': 1, 'optimizer': 'adam',  'learning_rate': 10**(-3),      'ridge': 10**(-4),      'patience': 6, 'n_epochs': 30}, # small net, moderate training
    {'id': 4, 'batch_size': 128,    'n_neurons': 32,    'n_hidden_layers': 2, 'optimizer': 'adam',  'learning_rate': 5*10**(-4),    'ridge': 10**(-4),      'patience': 8, 'n_epochs': 50}, # two hidden layers, modest
    {'id': 5, 'batch_size': 64,     'n_neurons': 64,    'n_hidden_layers': 2, 'optimizer': 'adam',  'learning_rate': 3*10**(-4),    'ridge': 10**(-3),      'patience': 6, 'n_epochs': 25}, # two layers, stronger regularization
    {'id': 6, 'batch_size': 256,    'n_neurons': 128,   'n_hidden_layers': 1, 'optimizer': 'sgd',   'learning_rate': 10**(-2),      'ridge': 10**(-4),      'patience': 5, 'n_epochs': 20}, # wider single hidden, larger batch
    {'id': 7, 'batch_size': 2048,   'n_neurons': 4,     'n_hidden_layers': 1, 'optimizer': 'adam',  'learning_rate': 10**(-3),      'ridge': 0,             'patience': 3, 'n_epochs': 30}, # very tiny net with large batches (fast)
    {'id': 8, 'batch_size': 256,    'n_neurons': 4,     'n_hidden_layers': 2, 'optimizer': 'sgd',   'learning_rate': 5*10**(-2),    'ridge': 5*10**(-4),    'patience': 6, 'n_epochs': 40}, # SGD-focused two small layers
    {'id': 9, 'batch_size': 512,    'n_neurons': 10,    'n_hidden_layers': 2, 'optimizer': 'adam',  'learning_rate': 10**(-3),      'ridge': 10**(-5),      'patience': 5, 'n_epochs': 40}, # balanced small MLP
]

## <span id="section-0" style="color:#00B8DE"> II - Precomputed Results </span>

Notice that we have already computed the results, so you can simply load the corresponding dataframes. Otherwise, skip this section and come back once you have generated the results.

In [5]:
df_put_bs = pd.read_csv('results_forward_im_profile/put_bs/bench_im_gauss_johnson.csv')
df_put_bs.sort_values('MSE test', inplace=True)
df_put_bs = df_put_bs.round(2)
df_put_bs = df_put_bs[df_put_bs['Id'].apply(lambda x: x[-2]) != '1']
list_display = ['GLSMC4', 'JPMMC301', 'JPMMC201', 'JPMMC203', 'JLSMC303', 'GLSMC0', 'JLSMC301', 'JLSMC101', 'JLSMC103', 'JLSMC001', 'JPMMC300', 'JPMMC200', 'JLSMC000', 'JLSMC200', 'JLSMC201']
df_put_bs = df_put_bs[df_put_bs['Id'].apply(lambda x: x in list_display)]
df_put_bs

,Id,Fit. type,Fit. params,Reg. type,MSE train,MSE test,Runtime
4,GLSMC4,MM,LR4,-,0.78,0.76,1s
63,JPMMC301,PM,10k,LR4,0.76,0.77,3mn9s
55,JPMMC201,PM,1k,LR4,0.90,0.92,29s
57,JPMMC203,PM,1k,NN3,0.98,0.98,28s
33,JLSMC303,MM,GLM4,NN3,1.19,1.23,5mn33s
0,GLSMC0,MM,LR2,-,1.30,1.30,0s
31,JLSMC301,MM,GLM4,LR4,1.40,1.49,6mn28s
15,JLSMC101,MM,LR4,LR4,2.00,2.10,3mn36s
17,JLSMC103,MM,LR4,NN3,2.16,2.23,3mn53s
7,JLSMC001,MM,LR2,LR4,2.23,2.34,1mn53s


In [6]:
df_put_bs_ml = pd.read_csv('results_forward_im_profile/put_bs/bench_im_neural_net.csv')
df_put_bs_ml.sort_values('MSE test', inplace=True)
df_put_bs_ml = df_put_bs_ml.round(2)
df_put_bs_ml

,Id,Architecture,Optimizer,Iterations,MSE train,MSE test,Runtime
4,ML4,2|32,adam|0.0005,50|128,0.08,0.08,26mn6s
5,ML5,2|64,adam|0.00030000000000000003,25|64,0.18,0.18,22mn27s
9,ML9,2|10,adam|0.001,40|512,0.23,0.24,11mn37s
1,ML1,1|10,adam|0.01,20|1024,0.24,0.24,5mn4s
8,ML8,2|4,sgd|0.05,40|256,0.51,0.53,8mn41s
0,ML0,1|4,sgd|0.01,15|1024,0.64,0.66,3mn59s
3,ML3,1|16,adam|0.001,30|256,0.74,0.77,12mn60s
7,ML7,1|4,adam|0.001,30|2048,1.31,1.42,12mn54s
2,ML2,1|8,adam|0.0005,20|512,1.62,1.81,9mn58s
6,ML6,1|128,sgd|0.01,20|256,3.08,3.61,7mn52s


In [7]:
df_swaption_hw1f = pd.read_csv('results_forward_im_profile/swaption_hw1f/bench_im_gauss_johnson.csv')
df_swaption_hw1f.sort_values('MSE test', inplace=True)
df_swaption_hw1f = df_swaption_hw1f.round(2)
df_swaption_hw1f = df_swaption_hw1f[df_swaption_hw1f['Id'].apply(lambda x: x[-2]) != '1']
list_display = ['JPMMC301', 'JPMMC200', 'JPMMC201', 'JLSMC103', 'JLSMC003', 'JPMMC003', 'GLSMC1', 'JLSMC100', 'JLSMC101', 'JPMMC203', 'JLSMC001', 'JLSMC000', 'JLSMC303', 'GLSMC0', 'JLSMC201']
df_swaption_hw1f = df_swaption_hw1f[df_swaption_hw1f['Id'].apply(lambda x: x in list_display)]
df_swaption_hw1f

,Id,Fit. type,Fit. params,Reg. type,MSE train,MSE test,Runtime
63,JPMMC301,PM,10k,LR4,3.10,2.66,12mn44s
54,JPMMC200,PM,1k,LR2,4.60,4.51,5mn4s
55,JPMMC201,PM,1k,LR4,5.59,5.00,4mn20s
17,JLSMC103,MM,LR4,NN3,18.53,17.12,1mn55s
9,JLSMC003,MM,LR2,NN3,19.68,18.75,2mn46s
41,JPMMC003,PM,1k,NN3,22.66,22.63,5mn19s
1,GLSMC1,MM,LR2,-,25.63,25.89,1s
14,JLSMC100,MM,LR4,LR2,32.21,31.28,1mn45s
15,JLSMC101,MM,LR4,LR4,50.30,40.63,1mn52s
57,JPMMC203,PM,1k,NN3,46.43,45.85,4mn27s


In [8]:
df_swaption_hw1f_ml = pd.read_csv('results_forward_im_profile/swaption_hw1f/bench_im_neural_net.csv')
df_swaption_hw1f_ml.sort_values('MSE test', inplace=True)
df_swaption_hw1f_ml = df_swaption_hw1f_ml.round(2)
df_swaption_hw1f_ml

,Id,Architecture,Optimizer,Iterations,MSE train,MSE test,Runtime
4,ML4,2|32,adam|0.0005,50|128,16.89,16.34,20mn1s
7,ML7,1|4,adam|0.001,30|2048,17.44,17.13,8mn56s
9,ML9,2|10,adam|0.001,40|512,18.83,17.37,9mn43s
2,ML2,1|8,adam|0.0005,20|512,22.21,19.55,10mn19s
5,ML5,2|64,adam|0.00030000000000000003,25|64,23.92,22.55,17mn52s
1,ML1,1|10,adam|0.01,20|1024,45.38,41.87,5mn30s
3,ML3,1|16,adam|0.001,30|256,56.47,50.52,10mn55s
0,ML0,1|4,sgd|0.01,15|1024,69.99,52.75,4mn18s
6,ML6,1|128,sgd|0.01,20|256,70.86,67.17,6mn56s
8,ML8,2|4,sgd|0.05,40|256,460.12,499.97,8mn44s


## <span id="section-0" style="color:#00B8DE"> III - European Put Option Black Scholes </span>

In [9]:
risk_factor = BlackScholes(0.3, 0.05, 100)
portfolio   = PutBlackScholes(95, maturity, risk_factor)
risk_factor.set_time_grid(time_grid)
risk_factor_paths_train = risk_factor.generate_paths(num_paths_train)
mtm_paths_train = portfolio.generate_paths(risk_factor_paths_train)
risk_factor_paths_test = risk_factor.generate_paths(num_paths_test)
mtm_paths_test = portfolio.generate_paths(risk_factor_paths_test)

groundtruth             = AnalyticalForwardInitialMargin(alpha, mpor, time_grid, portfolio)
im_true_paths_train     = groundtruth.generate_paths(risk_factor_paths_train, mtm_paths_train)
im_true_paths_test      = groundtruth.generate_paths(risk_factor_paths_test, mtm_paths_test)
im_true_profile_train   = groundtruth.generate_im_profile(im_true_paths_train)
im_true_profile_test    = groundtruth.generate_im_profile(im_true_paths_test)

### 1. GLSMC

In [10]:
list_id, list_fit_type, list_fit_params, list_reg_type, list_mse_train, list_mse_test, list_runtime = [], [], [], [], [], [], []
for moment_matching_setting_glsmc in moment_matching_setting_glsmc_list:
    list_fit_type.append('MM')
    list_fit_params.append(f'{moment_matching_setting_glsmc['method']}{moment_matching_setting_glsmc['order']}')
    list_reg_type.append('-')
    list_id.append(f'GLSMC{moment_matching_setting_glsmc['id']}')
    model = GaussianLeastSquaresMonteCarlo(alpha, mpor, time_grid, moment_matching_setting_glsmc)
    start_time = time.time()
    model.fit(mtm_paths_train)
    list_runtime.append(seconds_to_minutes(time.time() - start_time))
    im_paths_train      = model.generate_paths(mtm_paths_train)
    im_paths_test       = model.generate_paths(mtm_paths_test)
    list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
    list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
    im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
    im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    groundtruth.plot_im_profile(im_true_profile_train, axs[0])
    model.plot_im_profile(im_profile_train, axs[0])
    axs[0].set_xlabel('$t$')
    axs[0].set_ylabel('$\\text{IM}_t$')
    axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[0].set_title('training set')
    groundtruth.plot_im_profile(im_true_profile_test, axs[1])
    model.plot_im_profile(im_profile_test, axs[1])
    axs[1].set_xlabel('$t$')
    axs[1].set_ylabel('$\\text{IM}_t$')
    axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[1].set_title('testing set')

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
    plt.tight_layout()
    #plt.savefig(f'results_forward_im_profile/put_bs/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

### 2. JLSMC

In [ ]:
for moment_matching_setting_jlsmc in moment_matching_setting_jlsmc_list:
    for support_values_setting in support_values_setting_list:
        for quantile_function_setting in quantile_function_setting_list:
            list_fit_type.append('MM')
            list_fit_params.append(f'{moment_matching_setting_jlsmc['method']}{moment_matching_setting_jlsmc['order']}')
            reg_type = f'{quantile_function_setting['method']}{quantile_function_setting['order']}' if quantile_function_setting['method'] != 'kNN' else f'{quantile_function_setting['method'][1:]}{quantile_function_setting['n_neighbors']}'
            list_reg_type.append(reg_type)
            list_id.append(f'JLSMC{moment_matching_setting_jlsmc['id']}{support_values_setting['id']}{quantile_function_setting['id']}')
            model = JohnsonLeastSquaresMonteCarlo(alpha, mpor, time_grid, moment_matching_setting_jlsmc, support_values_setting, quantile_function_setting)
            start_time = time.time()
            model.fit(mtm_paths_train)
            list_runtime.append(seconds_to_minutes(time.time() - start_time))
            im_paths_train      = model.generate_paths(mtm_paths_train)
            im_paths_test       = model.generate_paths(mtm_paths_test)
            list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
            list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
            im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
            im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
            groundtruth.plot_im_profile(im_true_profile_train, axs[0])
            model.plot_im_profile(im_profile_train, axs[0])
            axs[0].set_xlabel('$t$')
            axs[0].set_ylabel('$\\text{IM}_t$')
            axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[0].set_title('training set')
            groundtruth.plot_im_profile(im_true_profile_test, axs[1])
            model.plot_im_profile(im_profile_test, axs[1])
            axs[1].set_xlabel('$t$')
            axs[1].set_ylabel('$\\text{IM}_t$')
            axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[1].set_title('testing set')

            handles, labels = axs[0].get_legend_handles_labels()
            fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
            plt.tight_layout()
            #plt.savefig(f'results_forward_im_profile/put_bs/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
            plt.close(fig)


### 3. JPMMC

In [ ]:
for percentile_matching_setting in percentile_matching_setting_list:
    for support_values_setting in support_values_setting_list:
        for quantile_function_setting in quantile_function_setting_list:
            list_fit_type.append('PM')
            list_fit_params.append(f'{int(percentile_matching_setting['Nnmc']/1000)}k')
            reg_type = f'{quantile_function_setting['method']}{quantile_function_setting['order']}' if quantile_function_setting['method'] != 'kNN' else f'{quantile_function_setting['method'][1:]}{quantile_function_setting['n_neighbors']}'
            list_reg_type.append(reg_type)
            list_id.append(f'JPMMC{percentile_matching_setting['id']}{support_values_setting['id']}{quantile_function_setting['id']}')
            model = JohnsonPercentileMatchingMonteCarlo(alpha, mpor, time_grid, percentile_matching_setting, support_values_setting, quantile_function_setting)
            start_time = time.time()
            model.fit(risk_factor_paths_train, mtm_paths_train, portfolio)
            list_runtime.append(seconds_to_minutes(time.time() - start_time))
            im_paths_train      = model.generate_paths(mtm_paths_train)
            im_paths_test       = model.generate_paths(mtm_paths_test)
            list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
            list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
            im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
            im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
            groundtruth.plot_im_profile(im_true_profile_train, axs[0])
            model.plot_im_profile(im_profile_train, axs[0])
            axs[0].set_xlabel('$t$')
            axs[0].set_ylabel('$\\text{IM}_t$')
            axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[0].set_title('training set')
            groundtruth.plot_im_profile(im_true_profile_test, axs[1])
            model.plot_im_profile(im_profile_test, axs[1])
            axs[1].set_xlabel('$t$')
            axs[1].set_ylabel('$\\text{IM}_t$')
            axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[1].set_title('testing set')

            handles, labels = axs[0].get_legend_handles_labels()
            fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
            plt.tight_layout()
            #plt.savefig(f'results_forward_im_profile/put_bs/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
            plt.close(fig)

In [ ]:
'''
df = pd.DataFrame({
    'Id': list_id,
    'Fit. type': list_fit_type,
    'Fit. params': list_fit_params,
    'Reg. type': list_reg_type,
    'MSE train': list_mse_train,
    'MSE test': list_mse_test,
    'Runtime': list_runtime
})

df.to_csv('results_forward_im_profile/put_bs/bench_im_gauss_johnson.csv', index=False)
'''

### 4. ML

In [ ]:
list_id, list_architecture, list_fit_optimizer, list_epochs, list_mse_train, list_mse_test, list_runtime = [], [], [], [], [], [], []
for nn_function_setting in nn_function_setting_list:
    list_architecture.append(f"{nn_function_setting['n_hidden_layers']}|{nn_function_setting['n_neurons']}")
    list_fit_optimizer.append(f"{nn_function_setting['optimizer']}|{nn_function_setting['learning_rate']}")
    list_epochs.append(f"{nn_function_setting['n_epochs']}|{nn_function_setting['batch_size']}")
    list_id.append(f'ML{nn_function_setting['id']}')
    model = NeuralQuantileRegression(alpha, mpor, time_grid, nn_function_setting)
    start_time = time.time()
    model.fit(risk_factor_paths_train, mtm_paths_train)
    list_runtime.append(seconds_to_minutes(time.time() - start_time))
    im_paths_train      = model.generate_paths(risk_factor_paths_train)
    im_paths_test       = model.generate_paths(risk_factor_paths_test)
    list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
    list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
    im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
    im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    groundtruth.plot_im_profile(im_true_profile_train, axs[0])
    model.plot_im_profile(im_profile_train, axs[0])
    axs[0].set_xlabel('$t$')
    axs[0].set_ylabel('$\\text{IM}_t$')
    axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[0].set_title('training set')
    groundtruth.plot_im_profile(im_true_profile_test, axs[1])
    model.plot_im_profile(im_profile_test, axs[1])
    axs[1].set_xlabel('$t$')
    axs[1].set_ylabel('$\\text{IM}_t$')
    axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[1].set_title('testing set')

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
    plt.tight_layout()
    #plt.savefig(f'results_forward_im_profile/put_bs/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

In [ ]:
'''
df = pd.DataFrame({
    'Id': list_id,
    'Architecture': list_architecture,
    'Optimizer': list_fit_optimizer,
    'Iterations': list_epochs,
    'MSE train': list_mse_train,
    'MSE test': list_mse_test,
    'Runtime': list_runtime
})

df.to_csv('results_forward_im_profile/put_bs/bench_im_neural_net.csv', index=False)
'''

## <span id="section-0" style="color:#00B8DE"> IV - Swaption One Factor Hull-White </span>

In [24]:
risk_factor = OneFactorHullWhite(0.01, 0.015, YieldCurve(0.05, -0.03, -0.18))
portfolio   = SwaptionOneFactorHullWhite(maturity, 0.04, TimeGrid(0.25, 4.75), 10000, risk_factor)
risk_factor.set_time_grid(time_grid)
risk_factor_paths_train = risk_factor.generate_paths(num_paths_train)
mtm_paths_train = portfolio.generate_paths(risk_factor_paths_train)
risk_factor_paths_test = risk_factor.generate_paths(num_paths_test)
mtm_paths_test = portfolio.generate_paths(risk_factor_paths_test)

groundtruth             = AnalyticalForwardInitialMargin(alpha, mpor, time_grid, portfolio)
im_true_paths_train     = groundtruth.generate_paths(risk_factor_paths_train, mtm_paths_train)
im_true_paths_test      = groundtruth.generate_paths(risk_factor_paths_test, mtm_paths_test)
im_true_profile_train   = groundtruth.generate_im_profile(im_true_paths_train)
im_true_profile_test    = groundtruth.generate_im_profile(im_true_paths_test)

### 1. GLSMC

In [ ]:
list_id, list_fit_type, list_fit_params, list_reg_type, list_mse_train, list_mse_test, list_runtime = [], [], [], [], [], [], []
for moment_matching_setting_glsmc in moment_matching_setting_glsmc_list:
    list_fit_type.append('MM')
    list_fit_params.append(f'{moment_matching_setting_glsmc['method']}{moment_matching_setting_glsmc['order']}')
    list_reg_type.append('-')
    list_id.append(f'GLSMC{moment_matching_setting_glsmc['id']}')
    model = GaussianLeastSquaresMonteCarlo(alpha, mpor, time_grid, moment_matching_setting_glsmc)
    start_time = time.time()
    model.fit(mtm_paths_train)
    list_runtime.append(seconds_to_minutes(time.time() - start_time))
    im_paths_train      = model.generate_paths(mtm_paths_train)
    im_paths_test       = model.generate_paths(mtm_paths_test)
    list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
    list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
    im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
    im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    groundtruth.plot_im_profile(im_true_profile_train, axs[0])
    model.plot_im_profile(im_profile_train, axs[0])
    axs[0].set_xlabel('$t$')
    axs[0].set_ylabel('$\\text{IM}_t$')
    axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[0].set_title('training set')
    groundtruth.plot_im_profile(im_true_profile_test, axs[1])
    model.plot_im_profile(im_profile_test, axs[1])
    axs[1].set_xlabel('$t$')
    axs[1].set_ylabel('$\\text{IM}_t$')
    axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[1].set_title('testing set')

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
    plt.tight_layout()
    #plt.savefig(f'results_forward_im_profile/swaption_hw1f/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

### 2. JLSMC

In [ ]:
for moment_matching_setting_jlsmc in moment_matching_setting_jlsmc_list:
    for support_values_setting in support_values_setting_list:
        for quantile_function_setting in quantile_function_setting_list:
            list_fit_type.append('MM')
            list_fit_params.append(f'{moment_matching_setting_jlsmc['method']}{moment_matching_setting_jlsmc['order']}')
            reg_type = f'{quantile_function_setting['method']}{quantile_function_setting['order']}' if quantile_function_setting['method'] != 'kNN' else f'{quantile_function_setting['method'][1:]}{quantile_function_setting['n_neighbors']}'
            list_reg_type.append(reg_type)
            list_id.append(f'JLSMC{moment_matching_setting_jlsmc['id']}{support_values_setting['id']}{quantile_function_setting['id']}')
            model = JohnsonLeastSquaresMonteCarlo(alpha, mpor, time_grid, moment_matching_setting_jlsmc, support_values_setting, quantile_function_setting)
            start_time = time.time()
            model.fit(mtm_paths_train)
            list_runtime.append(seconds_to_minutes(time.time() - start_time))
            im_paths_train      = model.generate_paths(mtm_paths_train)
            im_paths_test       = model.generate_paths(mtm_paths_test)
            list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
            list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
            im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
            im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
            groundtruth.plot_im_profile(im_true_profile_train, axs[0])
            model.plot_im_profile(im_profile_train, axs[0])
            axs[0].set_xlabel('$t$')
            axs[0].set_ylabel('$\\text{IM}_t$')
            axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[0].set_title('training set')
            groundtruth.plot_im_profile(im_true_profile_test, axs[1])
            model.plot_im_profile(im_profile_test, axs[1])
            axs[1].set_xlabel('$t$')
            axs[1].set_ylabel('$\\text{IM}_t$')
            axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[1].set_title('testing set')

            handles, labels = axs[0].get_legend_handles_labels()
            fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
            plt.tight_layout()
            #plt.savefig(f'results_forward_im_profile/swaption_hw1f/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
            plt.close(fig)

### 3. JPMMC

In [ ]:
for percentile_matching_setting in percentile_matching_setting_list:
    for support_values_setting in support_values_setting_list:
        for quantile_function_setting in quantile_function_setting_list:
            list_fit_type.append('PM')
            list_fit_params.append(f'{int(percentile_matching_setting['Nnmc']/1000)}k')
            reg_type = f'{quantile_function_setting['method']}{quantile_function_setting['order']}' if quantile_function_setting['method'] != 'kNN' else f'{quantile_function_setting['method'][1:]}{quantile_function_setting['n_neighbors']}'
            list_reg_type.append(reg_type)
            list_id.append(f'JPMMC{percentile_matching_setting['id']}{support_values_setting['id']}{quantile_function_setting['id']}')
            model = JohnsonPercentileMatchingMonteCarlo(alpha, mpor, time_grid, percentile_matching_setting, support_values_setting, quantile_function_setting)
            start_time = time.time()
            model.fit(risk_factor_paths_train, mtm_paths_train, portfolio)
            list_runtime.append(seconds_to_minutes(time.time() - start_time))
            im_paths_train      = model.generate_paths(mtm_paths_train)
            im_paths_test       = model.generate_paths(mtm_paths_test)
            list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
            list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
            im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
            im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

            fig, axs = plt.subplots(1, 2, figsize=(12, 4))
            groundtruth.plot_im_profile(im_true_profile_train, axs[0])
            model.plot_im_profile(im_profile_train, axs[0])
            axs[0].set_xlabel('$t$')
            axs[0].set_ylabel('$\\text{IM}_t$')
            axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[0].set_title('training set')
            groundtruth.plot_im_profile(im_true_profile_test, axs[1])
            model.plot_im_profile(im_profile_test, axs[1])
            axs[1].set_xlabel('$t$')
            axs[1].set_ylabel('$\\text{IM}_t$')
            axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
            axs[1].set_title('testing set')

            handles, labels = axs[0].get_legend_handles_labels()
            fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
            plt.tight_layout()
            #plt.savefig(f'results_forward_im_profile/swaption_hw1f/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
            plt.close(fig)

In [ ]:
'''
df = pd.DataFrame({
    'Id': list_id,
    'Fit. type': list_fit_type,
    'Fit. params': list_fit_params,
    'Reg. type': list_reg_type,
    'MSE train': list_mse_train,
    'MSE test': list_mse_test,
    'Runtime': list_runtime
})

df.to_csv('results_forward_im_profile/swaption_hw1f/bench_im_gauss_johnson.csv', index=False)
'''

### 4. ML

In [ ]:
list_id, list_architecture, list_fit_optimizer, list_epochs, list_mse_train, list_mse_test, list_runtime = [], [], [], [], [], [], []
for nn_function_setting in nn_function_setting_list:
    list_architecture.append(f"{nn_function_setting['n_hidden_layers']}|{nn_function_setting['n_neurons']}")
    list_fit_optimizer.append(f"{nn_function_setting['optimizer']}|{nn_function_setting['learning_rate']}")
    list_epochs.append(f"{nn_function_setting['n_epochs']}|{nn_function_setting['batch_size']}")
    list_id.append(f'ML{nn_function_setting['id']}')
    model = NeuralQuantileRegression(alpha, mpor, time_grid, nn_function_setting)
    start_time = time.time()
    model.fit(risk_factor_paths_train, mtm_paths_train)
    list_runtime.append(seconds_to_minutes(time.time() - start_time))
    im_paths_train      = model.generate_paths(risk_factor_paths_train)
    im_paths_test       = model.generate_paths(risk_factor_paths_test)
    list_mse_train.append(np.nanmean((im_paths_train - im_true_paths_train)**2))
    list_mse_test.append(np.nanmean((im_paths_test - im_true_paths_test)**2))
    im_profile_train    = groundtruth.generate_im_profile(im_paths_train)
    im_profile_test     = groundtruth.generate_im_profile(im_paths_test)

    fig, axs = plt.subplots(1, 2, figsize=(12, 4))
    groundtruth.plot_im_profile(im_true_profile_train, axs[0])
    model.plot_im_profile(im_profile_train, axs[0])
    axs[0].set_xlabel('$t$')
    axs[0].set_ylabel('$\\text{IM}_t$')
    axs[0].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[0].set_title('training set')
    groundtruth.plot_im_profile(im_true_profile_test, axs[1])
    model.plot_im_profile(im_profile_test, axs[1])
    axs[1].set_xlabel('$t$')
    axs[1].set_ylabel('$\\text{IM}_t$')
    axs[1].set_xlim(np.min(time_grid.grid)-time_grid.timestep, np.max(time_grid.grid)+time_grid.timestep)
    axs[1].set_title('testing set')

    handles, labels = axs[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.2))
    plt.tight_layout()
    #plt.savefig(f'results_forward_im_profile/swaption_hw1f/{list_id[-1]}.png', dpi=300, bbox_inches='tight')
    plt.close(fig)

In [ ]:
'''df = pd.DataFrame({
    'Id': list_id,
    'Architecture': list_architecture,
    'Optimizer': list_fit_optimizer,
    'Iterations': list_epochs,
    'MSE train': list_mse_train,
    'MSE test': list_mse_test,
    'Runtime': list_runtime
})

df.to_csv('results_forward_im_profile/swaption_hw1f/bench_im_neural_net.csv', index=False)'''